# Predicting KOSPI200

In [1]:
from datetime import datetime
import numpy as np
import pandas
import pandas as pd
from pandas import Series
import sys
sys.path.append('../lib/')
from dbConnect import *

C:\Users\trimu\Miniconda3\envs\py36_64\lib\site-packages\pymysql\cursors.py:165: Warning: (1366, "Incorrect string value: '\\xB4\\xEB\\xC7\\xD1\\xB9\\xCE...' for column 'VARIABLE_VALUE' at row 481")
  result = self._query(query)


In [2]:
end_date = '2016-12-31'
lags = 10
start_test = '2014-01-01'

In [3]:
code = "201"
df = pd.read_sql("SELECT * from marketcandle where code ='" + code + "' and date < '2016-12-31'" , con=engine)
df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'])
df.set_index('date', inplace=True)

In [4]:
df["returns"] = df["close"].pct_change()

In [5]:
df.head()

,code,open,high,low,close,volume,returns
date,,,,,,,
1998-07-27,201,37.84,38.33,37.15,37.93,54566,NaN
1998-07-28,201,37.96,38.70,37.96,38.70,42053,0.020301
1998-07-29,201,38.91,41.13,38.91,39.82,72961,0.028941
1998-07-30,201,39.78,40.12,39.12,39.89,66668,0.001758
1998-07-31,201,40.03,40.40,39.14,39.51,87472,-0.009526


In [6]:
df['returns'] = df["returns"].shift(-1)

In [7]:
df.head()

,code,open,high,low,close,volume,returns
date,,,,,,,
1998-07-27,201,37.84,38.33,37.15,37.93,54566,0.020301
1998-07-28,201,37.96,38.70,37.96,38.70,42053,0.028941
1998-07-29,201,38.91,41.13,38.91,39.82,72961,0.001758
1998-07-30,201,39.78,40.12,39.12,39.89,66668,-0.009526
1998-07-31,201,40.03,40.40,39.14,39.51,87472,-0.019489


Stochastic %K : $$\%K = \frac{C - L_{14}}{H_{14} - L_{14}} * 100 $$
C : 가장 최근 종가, L14: 이전 14 거래일 중 최저가, H14: 이전 14 거래일 중 최고가

In [8]:
tmpdf = df.copy(deep=True)

In [9]:
tmpdf['H14'] = Series.rolling(tmpdf['high'], window=14).max()
tmpdf['L14'] = Series.rolling(tmpdf['low'], window=14).min()
df['%K'] = (tmpdf['close'] - tmpdf['L14']) / (tmpdf['H14'] - tmpdf['L14']) * 100
df['%D'] = Series.rolling(df['%K'], window=3).mean()
df['slow%D'] = Series.rolling(df['%D'], window=3).mean()

Accumulation Distribution Oscillator (ADO) $$ADO = \frac{H_t - C_{t-1}}{H_t - L_t}$$ where C : t-1 의 close, $\quad L_t: low, \quad 
H_t: high$

In [10]:
tmpdf = df.copy(deep=True)

In [11]:
tmpdf['Ct-1'] = tmpdf['close'].shift(1) 

In [12]:
df['ADO'] = (tmpdf['high'] - tmpdf['Ct-1']) / (tmpdf['high'] - tmpdf['low'])

CCI(Commodity Channel Index) $$CCI = \frac{M_t - SM_t}{0.015D_t}$$ where $M_t = \frac {(H_t + L_t + C_t)}{3},\quad$ SM_t (n day moving average) $\quad \frac{\sum_{i=1}^{n}M_{t-i+1}}{n}, \quad D_t$(normal deviation) : $\frac{\sum_{i=1}^{n}\mid M_{t-i+1}-SM_t \mid}{n}$

+100: 상승돌파 - 단기매도,  하향돌파 - 단기매수

-100: 하향돌파 - 단기매도, 상향돌파 - 단기매수

In [13]:
tmpdf = df.copy(deep=True)

In [14]:
tmpdf['M'] = (tmpdf['high'] + tmpdf['low'] + tmpdf['close']) / 3
tmpdf['SM'] = tmpdf['M'].rolling(window=20).mean()
tmpdf['D'] =  (tmpdf['M'] - tmpdf['SM']).abs().rolling(window=20).mean()
df['CCI'] = (tmpdf['M'] - tmpdf['SM']) / (0.015 * tmpdf['D'])

Larry William's R% : $$\%R = 100 * \frac{H_r - C}{H_r - L_r}$$
r: 선정된 기간, C: 최근 종가, Hr: 선정된 기간의 고점, Lr: 선정된 기간의 저점

선정된 기간의 주가 범위와 현재의 종가를 측정하여 0 ~ 100% 사이에서 결정. 설정기간 7 일(주간차트)이 효과적. 매수과 매도 세력간의 종가관리 능력 측정

%R = (Highest High - Close)/(Highest High - Lowest Low) * -100

Lowest Low = lowest low for the look-back period

Highest High = highest high for the look-back period

%R is multiplied by -100 correct the inversion and move the decimal.


In [15]:
tmpdf = df.copy(deep=True)
tmpdf['Hr'] = Series.rolling(tmpdf['high'], window=7).max()
tmpdf['Lr'] =  Series.rolling(tmpdf['low'], window=7).min()
df['%R'] = 100 * (tmpdf['Hr'] - tmpdf['close']) / (tmpdf['Hr'] - tmpdf['Lr'])

In [16]:
tmpdf = df[['close']].copy(deep=True)

In [17]:
tmpdf['ewma12'] = Series.ewm(tmpdf['close'], span=12).mean()
tmpdf['ewma26'] = Series.ewm(tmpdf['close'], span=25).mean()
df['MACD'] = tmpdf['ewma12'] - tmpdf['ewma26']

Momentum = $$C_t - C_{t - n} $$

In [18]:
tslag = df[['close']].copy(deep=True)
for i in range(0, lags):
    tslag["Lag_" + str(i+1)] = tslag["close"].shift(i+1)

In [19]:
df['momentum'] = tslag['close'] - tslag['Lag_10']

ROC : Price Rate Of Change (일정시점의 가격 변화율을 백분율로 나타낸 것) $$ ROC = \frac{C_t}{C_{t-n}} * 100 $$

In [20]:
df['ROC'] = tslag['close'] / tslag['Lag_10'] * 100

RSI (Relative Strength Index) $$ RSI = 100 - (\frac{100}{1 + RS})$$ RS = $\frac{Average Gain}{Average Loss}$

In [21]:
tmpdf = df[['close']].copy()

delta = tmpdf['close'].diff()
delta = delta[1:]

up, down = delta.copy(), delta.copy()
up[up < 0] = 0
down[down > 0] = 0.0001

roll_up = up.ewm(span=14).mean()
roll_down = down.abs().ewm(span=14).mean()

RS = roll_up / roll_down
RSI = 100.0 - (100.0 / (1.0 + RS))
df['RSI'] = RSI

In [22]:
df['SMA10'] = Series.rolling(df['close'], window=10).mean()

In [23]:
df['WMA10'] = Series.ewm(df['close'], span=10).mean()

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as QDA

In [27]:
df.dropna(inplace=True)
df["Direction"] = np.sign(df["returns"])

In [29]:
X = df.drop(columns=['returns', 'Direction'])
y = df["Direction"]

In [31]:
# Create training and test sets
X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

In [32]:
# Create prediction DataFrame
pred = pd.DataFrame(index=y_test.index)

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
pred = (1.0 + y_pred * y_test)/2.0
hit_rate = np.mean(pred)
print('Logistic Regresstion {:.3f}'.format(hit_rate))

Logistic Regresstion 0.512


In [34]:
lda = LDA()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)
pred = (1.0 + y_pred * y_test)/2.0
hit_rate = np.mean(pred)
print('LDA {:.3f}'.format(hit_rate))

LDA 0.518


C:\Users\trimu\Miniconda3\envs\py36_64\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [35]:
models = QDA()
models.fit(X_train, y_train)
y_pred = models.predict(X_test)
pred = (1.0 + y_pred * y_test)/2.0
hit_rate = np.mean(pred)
print('QDA {:.3f}'.format(hit_rate))

QDA 0.518


C:\Users\trimu\Miniconda3\envs\py36_64\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


***